### Structured Streaming - práctica


#### Consultas sobre ficheiros - HDFS




In [ ]:
# Extrae un mínimo de 3 ficheiros de diferentes tamaños a partir de purchases.txt para realizar este exercicio
# compras1.tsv <- os 5.000 primeiros rexistros
# compras2.tsv <- 100.000 rexistros do medio do ficheiro orixinal (desde o 100.000 ao 200.000)
# compras3.tsv <- os 20.000 últimos rexistros

# Neste exercicio traballarás en HDFS

# Crea un ficheiro 'input_stream' no 'home' do teu HDFS (/user/hduser/input_stream)
# Copia o ficheiro compras1.tsv a 'input_stream'

In [ ]:
# 1. O noso sistema debe mostrar de xeito actualizado o total de vendas pagado cos diferentes métodos de pago
#    para cada día

# No primeiro momento estará compras1.txt -> espera a que se execute a consulta
# Move o segundo ficheiro e observa o resultado
# Move o terceiro documento e observa o resultado

# Compara a evolución dos datos ao ir procesando os diferentes ficheiros

# Observa a diferenza de executar a query con 'outputMode' = complete || update

In [ ]:
# Importa as librarías necesarias
from pyspark.sql.types import StructType

In [ ]:
# Definimos o esquema do Dataframe
userSchema = StructType() \
    .add("data", "date") \
    .add("hora", "string") \
    .add("tenda", "string") \
    .add("categoria", "string") \
    .add("venda", "float") \
    .add("pago", "string")

#userSchema

In [ ]:
# Creamos un Dataframe para ler o fluxo de ficheiros
tsvFiles = spark \
    .readStream \
    .option("sep", "\t") \
    .schema(userSchema) \
    .csv("input_stream")

In [ ]:
tsvFiles.createOrReplaceTempView('compras')
df_sql = spark.sql('SELECT data, pago, sum(venda) as total_vendas FROM compras GROUP BY data,pago')

In [ ]:
query = df_sql \
    .writeStream \
    .outputMode("complete") \
#    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination()